In [60]:
import pandas as pd
import numpy as np
import torch

In [61]:
import os 
if not os.path.isfile('data.zip'):
    !wget https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
    !unzip -n data.zip
    print('data downloaded.')
else:
    print('data already existed.')

data already existed.


In [62]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [63]:
dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

print(dec_train.shape, dec_val.shape, dec_test.shape)

(149, 203800) (149, 50950) (149, 139587)


In [64]:
def prepare_x(data):
    df = pd.DataFrame(data)
    df40 = df.iloc[:40, :].T
    
    new_column_names = []
    for i in range(1, 11):
        new_column_names.append(f"ask_price_level_{i}")
        new_column_names.append(f"ask_volume_level_{i}")
        new_column_names.append(f"bid_price_level_{i}")
        new_column_names.append(f"bid_volume_level_{i}")
    
    df40.columns = new_column_names
    return df40

In [65]:
x = prepare_x(dec_train)
x.head(10)

,ask_price_level_1,ask_volume_level_1,bid_price_level_1,bid_volume_level_1,ask_price_level_2,ask_volume_level_2,bid_price_level_2,bid_volume_level_2,ask_price_level_3,ask_volume_level_3,...,bid_price_level_8,bid_volume_level_8,ask_price_level_9,ask_volume_level_9,bid_price_level_9,bid_volume_level_9,ask_price_level_10,ask_volume_level_10,bid_price_level_10,bid_volume_level_10
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,0.2591,0.00134,0.2629,0.00146,0.2588,0.00123,0.2633,0.00311,0.2579,0.00128
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,0.2593,0.00143,0.2637,0.00165,0.2591,0.00134,0.2646,0.00138,0.2588,0.00123
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,0.2593,0.00143,0.2629,0.00146,0.2591,0.00134,0.2633,0.00311,0.2588,0.00123
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,0.2591,0.00134,0.2637,0.00165,0.2588,0.00123,0.2646,0.00138,0.2579,0.00128
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,0.2591,0.00134,0.2637,0.00165,0.2588,0.00123,0.2646,0.00138,0.2579,0.00128
5,0.2614,0.00322,0.2607,0.00209,0.2617,0.00577,0.2606,0.00426,0.2619,0.00850,...,0.2593,0.00143,0.2637,0.00165,0.2591,0.00134,0.2646,0.00138,0.2589,0.00081
6,0.2614,0.00012,0.2607,0.00409,0.2616,0.00259,0.2606,0.00326,0.2617,0.00818,...,0.2593,0.00143,0.2633,0.00311,0.2591,0.00134,0.2637,0.00165,0.2589,0.00081
7,0.2617,0.00918,0.2611,0.00400,0.2619,0.00850,0.2609,0.00169,0.2620,0.00138,...,0.2599,0.00159,0.2646,0.00138,0.2595,0.00100,0.2650,0.01500,0.2593,0.00143
8,0.2617,0.00818,0.2611,0.00400,0.2618,0.00115,0.2609,0.00169,0.2619,0.00850,...,0.2600,0.00500,0.2646,0.00138,0.2599,0.00159,0.2650,0.01500,0.2596,0.00138
9,0.2617,0.00818,0.2612,0.00500,0.2618,0.00115,0.2611,0.00400,0.2619,0.00850,...,0.2600,0.00500,0.2637,0.00165,0.2599,0.00159,0.2646,0.00138,0.2596,0.00138


In [76]:
def compute_volatility(df, ks=[10, 20, 30, 50, 100]):
    """
    Compute volatilities of the midprice for the level 1 over a range of k precedent rows.

    Parameters:
    df (pd.DataFrame): DataFrame containing price and volume data.
    ks (list of int): List of window sizes for which volatilities are computed.

    Returns:
    pd.DataFrame: DataFrame with columns for volatilities for each k in ks.
    """
    # Compute the mid-price for level 1
    df['mid_price_level_1'] = (df['ask_price_level_1'] + df['bid_price_level_1']) / 2
    
    # Compute the price change (difference in mid-price from the previous row)
    df['price_change_level_1'] = df['mid_price_level_1'].diff()
    
    # Create a DataFrame to store volatilities for each k
    vol_df = pd.DataFrame(index=df.index)
    
    for k in ks:
        # Calculate the rolling standard deviation (volatility) over the last k rows
        vol_df[f'volatility_k={k}'] = df['price_change_level_1'].rolling(window=k).std()
    
    return vol_df

In [78]:
ks=[10, 20, 30, 50, 100]
y = compute_volatility(x, ks=ks)
y[100:110]

,volatility_k=10,volatility_k=20,volatility_k=30,volatility_k=50,volatility_k=100
100,0.000078,0.000089,0.000082,0.000078,0.000080
101,0.000078,0.000088,0.000082,0.000078,0.000080
102,0.000078,0.000088,0.000080,0.000074,0.000079
103,0.000071,0.000065,0.000080,0.000074,0.000080
104,0.000037,0.000067,0.000081,0.000073,0.000080
105,0.000037,0.000067,0.000079,0.000073,0.000080
106,0.000042,0.000071,0.000080,0.000074,0.000081
107,0.000041,0.000071,0.000080,0.000074,0.000073
108,0.000041,0.000062,0.000080,0.000073,0.000073
109,0.000064,0.000071,0.000083,0.000075,0.000074
